[code is from this link](https://www.analyticsvidhya.com/blog/2022/01/complete-guide-to-anomaly-detection-with-autoencoders-using-tensorflow/)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler


mpl.rcParams['figure.figsize'] = (10, 5)
mpl.rcParams['axes.grid'] = False

## DATA

In [ ]:
!cat "/ECG5000_TRAIN.txt" "/ECG5000_TEST.txt" > ecg_final.txt
df = pd.read_csv("ecg_final.txt", sep='  ', header=None)
df.shape

In [ ]:
df = df.add_prefix('c')
df['c0'].value_counts()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.values, df.values[:, 0:1], test_size=0.2, random_state=111)


In [ ]:
scaler = MinMaxScaler()
data_scaled = scaler.fit(x_train)
train_data_scaled = data_scaled.transform(x_train)
test_data_scaled = data_scaled.transform(x_test)

In [ ]:
normal_train_data = pd.DataFrame(train_data_scaled).add_prefix('c').query('c0 == 0').values[:, 1:]
anomaly_train_data = pd.DataFrame(train_data_scaled).add_prefix('c').query('c0 > 0').values[:, 1:]
normal_test_data = pd.DataFrame(test_data_scaled).add_prefix('c').query('c0 == 0').values[:, 1:]
anomaly_test_data = pd.DataFrame(test_data_scaled).add_prefix('c').query('c0 > 0').values[:, 1:]

In [ ]:
plt.plot(normal_train_data[0])
plt.plot(normal_train_data[1])
plt.plot(normal_train_data[2])
plt.title("Normal Data")
plt.show()

In [ ]:
plt.plot(anomaly_train_data[0])
plt.plot(anomaly_train_data[1])
plt.plot(anomaly_train_data[2])
plt.title("Anomaly Data")
plt.show()

## MODEL

In [ ]:
# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Dense(64, activation="relu"))
# model.add(tf.keras.layers.Dense(32, activation="relu"))
# model.add(tf.keras.layers.Dense(16, activation="relu"))
# model.add(tf.keras.layers.Dense(8, activation="relu"))
# model.add(tf.keras.layers.Dense(16, activation="relu"))
# model.add(tf.keras.layers.Dense(32, activation="relu"))
# model.add(tf.keras.layers.Dense(64, activation="relu"))
# model.add(tf.keras.layers.Dense(140, activation="sigmoid"))

class AutoEncoder(Model):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        f0, f1, f2, f3, f4, f5, f6 = 8, 16, 32, 64, 128, 256, 512
        self.encoder = tf.keras.Sequential([tf.keras.layers.Dense(f3, activation="relu"),
                                            tf.keras.layers.Dense(f2, activation="relu"),
                                            tf.keras.layers.Dense(f1, activation="relu"),
                                            tf.keras.layers.Dense(f0, activation="relu")
                                            ])
        self.decoder = tf.keras.Sequential([tf.keras.layers.Dense(f1, activation="relu"),
                                            tf.keras.layers.Dense(f2, activation="relu"),
                                            tf.keras.layers.Dense(f3, activation="relu"),
                                            tf.keras.layers.Dense(140, activation="sigmoid")
                                            ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
model = AutoEncoder()
model.compile(optimizer='adam', loss="mae")

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, mode="min")
history = model.fit(normal_train_data, normal_train_data, epochs=50, batch_size=120,
                    validation_data=(train_data_scaled[:, 1:], train_data_scaled[:, 1:]),
                    shuffle=True,
                    callbacks=[early_stopping,
                               ]
                    )


## EVAL

In [ ]:
encoder_out = model.encoder(normal_test_data).numpy()  #8 unit representation of data
decoder_out = model.decoder(encoder_out).numpy()

plt.plot(normal_test_data[0], 'b')
plt.plot(decoder_out[0], 'r')
plt.title("Model performance on Normal data")
plt.show()

In [ ]:
encoder_out_a = model.encoder(anomaly_test_data).numpy()  #8 unit representation of data
decoder_out_a = model.decoder(encoder_out_a).numpy()

plt.plot(anomaly_test_data[0], 'b')
plt.plot(decoder_out_a[0], 'r')
plt.title("Model performance on Anomaly Data")
plt.show()

In [ ]:
reconstruction = model.predict(normal_test_data)
train_loss = tf.keras.losses.mae(reconstruction, normal_test_data)
plt.hist(train_loss, bins=50)

In [ ]:
threshold = np.mean(train_loss)+2*np.std(train_loss)
reconstruction_a = model.predict(anomaly_test_data)
train_loss_a = tf.keras.losses.mae(reconstruction_a, anomaly_test_data)

plt.hist(train_loss_a, bins=50)
plt.title("loss on anomaly test data")
plt.show()

In [ ]:
plt.hist(train_loss, bins=50, label='normal')
plt.hist(train_loss_a, bins=50, label='anomaly')
plt.axvline(threshold, color='r', linewidth=3, linestyle='dashed', label='{:0.3f}'.format(threshold))
plt.legend(loc='upper right')
plt.title("Normal and Anomaly Loss")
plt.show()

In [ ]:
preds = tf.math.less(train_loss, threshold)
tf.math.count_nonzero(preds)

In [ ]:
preds_a = tf.math.greater(train_loss_a, threshold)
tf.math.count_nonzero(preds_a)